In [9]:
import os
# import pkg_resources
# pkg_resources.require('Torch==2.0.1')
!pip install torch==2.0.1
import torch
print('Using torch', torch.__version__)
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install ogb
!pip install rdkit-pypi

Using torch 2.0.1+cu117
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html


In [10]:
# FPGNN

from argparse import Namespace
import torch
import torch.nn as nn
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys
# from fpgnn.data import GetPubChemFPs, create_graph, get_atom_features_dim
import csv
import re

# required functions for our model

def remove_non_alphanumeric(input_string):
    # Use a regular expression to match only letters and numbers
    alphanumeric_pattern = re.compile(r'[\W_]+')

    # Replace non-alphanumeric characters with an empty string
    result_string = alphanumeric_pattern.sub('', input_string)

    return result_string

atom_type_max = 100
atom_f_dim = 133
atom_features_define = {
    'atom_symbol': list(range(atom_type_max)),
    'degree': [0, 1, 2, 3, 4, 5],
    'formal_charge': [-1, -2, 1, 2, 0],
    'charity_type': [0, 1, 2, 3],
    'hydrogen': [0, 1, 2, 3, 4],
    'hybridization': [
        Chem.rdchem.HybridizationType.SP,
        Chem.rdchem.HybridizationType.SP2,
        Chem.rdchem.HybridizationType.SP3,
        Chem.rdchem.HybridizationType.SP3D,
        Chem.rdchem.HybridizationType.SP3D2
    ],}

smile_changed = {}

def get_atom_features_dim():
    return atom_f_dim

def onek_encoding_unk(key,length):
    encoding = [0] * (len(length) + 1)
    index = length.index(key) if key in length else -1
    encoding[index] = 1

    return encoding

def get_atom_feature(atom):
    feature = onek_encoding_unk(atom.GetAtomicNum() - 1, atom_features_define['atom_symbol']) + \
           onek_encoding_unk(atom.GetTotalDegree(), atom_features_define['degree']) + \
           onek_encoding_unk(atom.GetFormalCharge(), atom_features_define['formal_charge']) + \
           onek_encoding_unk(int(atom.GetChiralTag()), atom_features_define['charity_type']) + \
           onek_encoding_unk(int(atom.GetTotalNumHs()), atom_features_define['hydrogen']) + \
           onek_encoding_unk(int(atom.GetHybridization()), atom_features_define['hybridization']) + \
           [1 if atom.GetIsAromatic() else 0] + \
           [atom.GetMass() * 0.01]
    return feature

class GraphOne:
    def __init__(self,smile,args):
        self.smile = smile
        self.atom_feature = []

        mol = Chem.MolFromSmiles(self.smile)
        # condition is added by me
        if mol is None:
          # print('baddddddd')
          smile = 'N#CCC1=CC=CC=C1'
          self.smile = smile
          mol = Chem.MolFromSmiles(self.smile)
        else:
          # print('good')
          blood = 7
        self.atom_num = mol.GetNumAtoms()

        for i, atom in enumerate(mol.GetAtoms()):
            self.atom_feature.append(get_atom_feature(atom))
        self.atom_feature = [self.atom_feature[i] for i in range(self.atom_num)]

class GraphBatch:
    def __init__(self,graphs,args):
        smile_list = []
        for graph in graphs:
            smile_list.append(graph.smile)
        self.smile_list = smile_list
        self.smile_num = len(self.smile_list)
        self.atom_feature_dim = get_atom_features_dim()
        self.atom_no = 1
        self.atom_index = []

        atom_feature = [[0]*self.atom_feature_dim]
        for graph in graphs:
            atom_feature.extend(graph.atom_feature)
            self.atom_index.append((self.atom_no,graph.atom_num))
            self.atom_no += graph.atom_num

        self.atom_feature = torch.FloatTensor(atom_feature)

    def get_feature(self):
        return self.atom_feature,self.atom_index

def create_graph(smile,args):
    graphs = []
    for one in smile:
        if one in smile_changed:
            graph = smile_changed[one]
        else:
            graph = GraphOne(one, args)
            smile_changed[one] = graph
        graphs.append(graph)
    return GraphBatch(graphs,args)

# our model is shown below

atts_out = []

class FPN(nn.Module):
    def __init__(self,args):
        super(FPN, self).__init__()
        self.fp_2_dim=args.fp_2_dim
        self.dropout_fpn = args.dropout
        self.cuda = args.cuda
        self.hidden_dim = args.hidden_size
        self.args = args
        if hasattr(args,'fp_type'):
            self.fp_type = args.fp_type
        else:
            self.fp_type = 'mixed'

        if self.fp_type == 'mixed':
            self.fp_dim = 1489
        else:
            self.fp_dim = 1024

        if hasattr(args,'fp_changebit'):
            self.fp_changebit = args.fp_changebit
        else:
            self.fp_changebit = None

        self.fc1=nn.Linear(self.fp_dim, self.fp_2_dim)
        self.act_func = nn.ReLU()
        self.fc2 = nn.Linear(self.fp_2_dim, self.hidden_dim)
        self.dropout = nn.Dropout(p=self.dropout_fpn)

    def forward(self, smile):
        fp_list=[]
        for i, one in enumerate(smile):
            fp=[]
            mol = Chem.MolFromSmiles(one)

            if self.fp_type == 'mixed':
                fp_maccs = AllChem.GetMACCSKeysFingerprint(mol)
                fp_phaErGfp = AllChem.GetErGFingerprint(mol,fuzzIncrement=0.3,maxPath=21,minPath=1)
                # fp_pubcfp = GetPubChemFPs(mol)
                fp.extend(fp_maccs)
                fp.extend(fp_phaErGfp)
                # fp.extend(fp_pubcfp)
            else:
                fp_morgan = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
                fp.extend(fp_morgan)
            fp_list.append(fp)

        if self.fp_changebit is not None and self.fp_changebit != 0:
            fp_list = np.array(fp_list)
            fp_list[:,self.fp_changebit-1] = np.ones(fp_list[:,self.fp_changebit-1].shape)
            fp_list.tolist()

        fp_list = torch.Tensor(fp_list)

        if self.cuda:
            fp_list = fp_list.cuda()
        fpn_out = self.fc1(fp_list)
        fpn_out = self.dropout(fpn_out)
        fpn_out = self.act_func(fpn_out)
        fpn_out = self.fc2(fpn_out)
        return fpn_out

class GATLayer(nn.Module):

    def __init__(self, in_features, out_features, dropout_gnn, alpha, inter_graph, concat=True):
        super(GATLayer, self).__init__()
        self.dropout_gnn= dropout_gnn
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat
        self.dropout = nn.Dropout(p=self.dropout_gnn)
        self.inter_graph = inter_graph

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)
        if self.inter_graph is not None:
            self.atts_out = []

    def forward(self,mole_out,adj):
        atom_feature = torch.mm(mole_out, self.W)
        N = atom_feature.size()[0]

        atom_trans = torch.cat([atom_feature.repeat(1, N).view(N * N, -1), atom_feature.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e = self.leakyrelu(torch.matmul(atom_trans, self.a).squeeze(2))

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)

        if self.inter_graph is not None:
            att_out = attention
            if att_out.is_cuda:
                att_out = att_out.cpu()
            att_out = np.array(att_out)
            att_out[att_out<-10000] = 0
            att_out = att_out.tolist()
            atts_out.append(att_out)

        attention = nn.functional.softmax(attention, dim=1)
        attention = self.dropout(attention)
        output = torch.matmul(attention, atom_feature)

        if self.concat:
            return nn.functional.elu(output)
        else:
            return output


class GATOne(nn.Module):
    def __init__(self,args):
        super(GATOne, self).__init__()
        self.nfeat = get_atom_features_dim()
        self.nhid = args.nhid
        self.dropout_gnn = args.dropout_gat
        self.atom_dim = args.hidden_size
        self.alpha = 0.2
        self.nheads = args.nheads
        self.args = args
        self.dropout = nn.Dropout(p=self.dropout_gnn)

        if hasattr(args,'inter_graph'):
            self.inter_graph = args.inter_graph
        else:
            self.inter_graph = None

        self.attentions = [GATLayer(self.nfeat, self.nhid, dropout_gnn=self.dropout_gnn, alpha=self.alpha, inter_graph=self.inter_graph, concat=True) for _ in range(self.nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = GATLayer(self.nhid * self.nheads, self.atom_dim, dropout_gnn=self.dropout_gnn, alpha=self.alpha, inter_graph=self.inter_graph, concat=False)

    def forward(self,mole_out,adj):
        mole_out = self.dropout(mole_out)
        mole_out = torch.cat([att(mole_out, adj) for att in self.attentions], dim=1)
        mole_out = self.dropout(mole_out)
        mole_out = nn.functional.elu(self.out_att(mole_out, adj))
        return nn.functional.log_softmax(mole_out, dim=1)

class GATEncoder(nn.Module):
    def __init__(self,args):
        super(GATEncoder,self).__init__()
        self.cuda = args.cuda
        self.args = args
        self.encoder = GATOne(self.args)

    def forward(self,mols,smiles):
        atom_feature, atom_index = mols.get_feature()
        if self.cuda:
            atom_feature = atom_feature.cuda()

        gat_outs=[]

        for i,one in enumerate(smiles):
            adj = []
            mol = Chem.MolFromSmiles(one)
            # added by me
            if mol is None:
              one = 'N#CCC1=CC=CC=C1'
              mol = Chem.MolFromSmiles(one)
            adj = Chem.rdmolops.GetAdjacencyMatrix(mol)
            # print('broooooooo')
            adj = adj/1
            adj = torch.from_numpy(adj)
            if self.cuda:
                adj = adj.cuda()

            atom_start, atom_size = atom_index[i]
            one_feature = atom_feature[atom_start:atom_start+atom_size]

            gat_atoms_out = self.encoder(one_feature,adj)
            gat_out = gat_atoms_out.sum(dim=0)/atom_size
            gat_outs.append(gat_out)
        gat_outs = torch.stack(gat_outs, dim=0)
        return gat_outs

class GAT(nn.Module):
    def __init__(self,args):
        super(GAT,self).__init__()
        self.args = args
        self.encoder = GATEncoder(self.args)

    def forward(self,smile):
        mol = create_graph(smile, self.args)
        gat_out = self.encoder.forward(mol,smile)

        return gat_out

class FpgnnModel(nn.Module):
    def __init__(self,is_classif,gat_scale,cuda,dropout_fpn):
        super(FpgnnModel, self).__init__()
        self.gat_scale = gat_scale
        self.is_classif = is_classif
        self.cuda = cuda
        self.dropout_fpn = dropout_fpn
        if self.is_classif:
            self.sigmoid = nn.Sigmoid()

    def create_gat(self,args):
        self.encoder3 = GAT(args)

    def create_fpn(self,args):
        self.encoder2 = FPN(args)

    def create_scale(self,args):
        linear_dim = int(args.hidden_size)
        if self.gat_scale == 1:
            self.fc_gat = nn.Linear(linear_dim,linear_dim)
        elif self.gat_scale == 0:
            self.fc_fpn = nn.Linear(linear_dim,linear_dim)
        else:
            self.gat_dim = int((linear_dim*2*self.gat_scale)//1)
            self.fc_gat = nn.Linear(linear_dim,self.gat_dim)
            self.fc_fpn = nn.Linear(linear_dim,linear_dim*2-self.gat_dim)
        self.act_func = nn.ReLU()

    def create_ffn(self,args):
        linear_dim = args.hidden_size
        if self.gat_scale == 1:
            self.ffn = nn.Sequential(
                                     nn.Dropout(self.dropout_fpn),
                                     nn.Linear(in_features=linear_dim, out_features=linear_dim, bias=True),
                                     nn.ReLU(),
                                     nn.Dropout(self.dropout_fpn),
                                     nn.Linear(in_features=linear_dim, out_features=args.task_num, bias=True)
                                     )
        elif self.gat_scale == 0:
            self.ffn = nn.Sequential(
                                     nn.Dropout(self.dropout_fpn),
                                     nn.Linear(in_features=linear_dim, out_features=linear_dim, bias=True),
                                     nn.ReLU(),
                                     nn.Dropout(self.dropout_fpn),
                                     nn.Linear(in_features=linear_dim, out_features=args.task_num, bias=True)
                                     )

        else:
            self.ffn = nn.Sequential(
                                     nn.Dropout(self.dropout_fpn),
                                     nn.Linear(in_features=linear_dim*2, out_features=linear_dim, bias=True),
                                     nn.ReLU(),
                                     nn.Dropout(self.dropout_fpn),
                                     nn.Linear(in_features=linear_dim, out_features=args.task_num, bias=True)
                                     )

    def forward(self,input):
        if self.gat_scale == 1:
            output = self.encoder3(input)
        elif self.gat_scale == 0:
            output = self.encoder2(input)
        else:
            gat_out = self.encoder3(input)
            fpn_out = self.encoder2(input)
            gat_out = self.fc_gat(gat_out)
            gat_out = self.act_func(gat_out)

            fpn_out = self.fc_fpn(fpn_out)
            fpn_out = self.act_func(fpn_out)

            output = torch.cat([gat_out,fpn_out],axis=1)
        output = self.ffn(output)

        if self.is_classif and not self.training:
            output = self.sigmoid(output)

        return output

def get_atts_out():
    return atts_out

def FPGNN(args):
    if args.dataset_type == 'classification':
        is_classif = 1
    else:
        is_classif = 0
    model = FpgnnModel(is_classif,args.gat_scale,args.cuda,args.dropout)
    if args.gat_scale == 1:
        model.create_gat(args)
        model.create_ffn(args)
    elif args.gat_scale == 0:
        model.create_fpn(args)
        model.create_ffn(args)
    else:
        model.create_gat(args)
        model.create_fpn(args)
        model.create_scale(args)
        model.create_ffn(args)

    for param in model.parameters():
        if param.dim() == 1:
            nn.init.constant_(param, 0)
        else:
            nn.init.xavier_normal_(param)

    return model

In [11]:
import matplotlib.pyplot as plt

def plot_training_metrics(losses, val_metrics, test_metrics, epoch_times):
    """
    Plot training loss, validation metric, and test metric over epochs.

    Parameters:
    - losses: List of training losses for each epoch.
    - val_metrics: List of validation metrics for each epoch.
    - test_metrics: List of test metrics for each epoch.
    """
    epochs = range(1, len(losses) + 1)

    # Plotting training loss
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, losses, label='Training Loss', marker='o')
    plt.title('Training Loss vs. Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plotting validation and test metrics
    plt.subplot(1, 2, 2)
    plt.plot(epochs, val_metrics, label='Validation Metric', marker='o')
    plt.plot(epochs, test_metrics, label='Test Metric', marker='o')
    plt.title('Validation and Test Metrics vs. Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Metric')
    plt.legend()

    # Plotting time vs. epoch
    plt.subplot(1, 3, 3)
    plt.plot(epochs, epoch_times, label='Time per Epoch', marker='o', color='orange')
    plt.title('Epoch Time vs. Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Time (seconds)')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Example usage:
# plot_training_metrics(training_losses, validation_metrics, test_metrics)

In [12]:
from argparse import Namespace
from logging import Logger
import os
import csv
import numpy as np
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR


#         model.zero_grad()
#         pred = model(smile)
#         loss = loss_f(pred,target) * weight * mask
#         loss = loss.sum() / mask.sum()
#         loss_sum += loss.item()
#         data_used += len(smile)
#         loss.backward()
#         optimizer.step()
#         if isinstance(scheduler, NoamLR):
#             scheduler.step()
#     if isinstance(scheduler, ExponentialLR):
#         scheduler.step()



def use_fpgnn(dataset_name, task_type):

  # import torch_geometric
  # from torch_geometric.datasets import TUDataset
  # from torch_geometric.loader import DataLoader
  # from torch_geometric.nn import GCNConv, GraphMultisetTransformer
  import torch
  from torch_geometric.datasets import MoleculeNet
  torch.autograd.set_detect_anomaly(True)
  # import time

  import os.path as osp
  import time
  from tqdm import tqdm

  import torch
  import torch.nn.functional as F
  from torch.nn import Linear

  from torch_geometric.datasets import TUDataset
  # from torch_geometric.loader import DataLoader
  from torch.utils.data import Dataset, DataLoader, random_split
  from torch_geometric.nn import GCNConv, GraphMultisetTransformer
  import torch_geometric

  ### Importing OGB dataset
  from ogb.graphproppred.dataset_pyg import PygGraphPropPredDataset
  from ogb.graphproppred import Evaluator

  0% 0/54 [00:00<?, ?it/s]

loss: tensor(38986.0391, grad_fn=<DivBackward1>)


  2% 1/54 [00:18<16:01, 18.14s/it]

loss: tensor(29129.2695, grad_fn=<DivBackward1>)


  4% 2/54 [00:27<11:11, 12.91s/it]

...

loss: tensor(21154.1113, grad_fn=<DivBackward1>)


 39% 21/54 [02:19<03:49,  6.95s/it]Exception ignored in: <function _xla_gc_callback at 0x7efc55dd36d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


loss: tensor(20683.3086, grad_fn=<DivBackward1>)


 41% 22/54 [02:30<03:38,  6.82s/it]


KeyboardInterrupt: ignored

In [19]:
def run_fpgnn(dataset_name, task_name):
  import torch
  import torch.nn as nn
  import torch.optim as optim
  from torch.utils.data import DataLoader
  import torch
  from torch_geometric.datasets import MoleculeNet
  import time
  from torch.utils.data import DataLoader, random_split
  from torch_geometric.data import Batch
  from tqdm import tqdm
  import numpy as np

  dataset = MoleculeNet('\tmp\mol', dataset_name).shuffle()
  ### Importing OGB dataset
  from ogb.graphproppred.dataset_pyg import PygGraphPropPredDataset
  from ogb.graphproppred import Evaluator
  # dataset2= = PygGraphPropPredDataset(name=task_name)
  evaluator = Evaluator(task_name)
  # Replace YourDataset with your actual dataset class
  # from your_module import YourDataset

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  args = Namespace(dataset_type="classification", gat_scale=1, cuda=False, dropout=0.25, nhid=128, dropout_gat=0.25, hidden_size=128, nheads=4,
                    task_num=617)
  model = FPGNN(args).to(device)

  optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
  batch_size=128
  train_size = int(0.8 * len(dataset))
  val_size = (len(dataset) - train_size) // 2
  test_size = len(dataset) - train_size - val_size

  # Split the dataset into train, validation, and test sets
  train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

  def collate(data_list):
      """
      Custom collate function to handle torch_geometric.data.data.Data instances.
      Converts a list of Data instances to a Batch.
      """
      return Batch.from_data_list(data_list)


  # Create data loaders with the custom collate function
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)

  clf_criterion = torch.nn.BCEWithLogitsLoss(torch.tensor(10))
  reg_criterion = torch.nn.MSELoss()




  def train(model, train_loader, criterion, optimizer, device):
      model.train()
      total_loss = 0.0

      epoch_iter = tqdm(train_loader, ncols=0)
      for step, data in enumerate(epoch_iter):
          data = data.to(device)
          smiles, labels = data.smiles, data.y

          optimizer.zero_grad()

          # Forward pass
          output = model(smiles)

          # Compute the loss
          is_labeled = ~torch.isnan(data.y) # Mask NaNs
          loss = criterion(output[is_labeled], data.y[is_labeled])

          # Backward pass and optimization
          loss.backward()
          # prevent exploding gradients
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
          optimizer.step()

          total_loss += loss.item()

      return total_loss / len(train_loader)

  @torch.no_grad()
  def test(model, test_loader, criterion, device):

      model.eval()

      y_true = []
      y_pred = []

      total_correct = 0
      epoch_iter = tqdm(test_loader, ncols=0)
      for step, batch in enumerate(epoch_iter):
          batch = batch.to(device)
          smiles, labels = batch.smiles, batch.y

          out = model(batch.smiles)
          output = out.cpu().numpy()
          print(out)
          print(batch.y)
          y_true.append(batch.y.view(out.shape).detach().cpu())
          y_pred.append(out.detach().cpu())
          total_correct += int((torch.argmax(out.to(float), dim=1) == torch.argmax(batch.y.to(float), dim=1)).sum())
      y_true = torch.cat(y_true, dim = 0).numpy()
      y_pred = torch.cat(y_pred, dim = 0).numpy()
      input_dict = {"y_true": y_true, "y_pred": y_pred}
      return evaluator.eval(input_dict)

  times = []
  training_losses = []
  validation_metrics = []
  test_metrics = []
  for epoch in range(1, 6):
      start = time.time()
      train_loss = train(model, train_loader, clf_criterion, optimizer, device)
      val_acc = test(model, val_loader, clf_criterion, device)
      test_acc = test(model, test_loader, clf_criterion, device)
      training_losses.append(train_loss)
      validation_metrics.append(val_acc)
      test_metrics.append(test_acc)
      print(f'Epoch: {epoch:03d}, Loss: {train_loss:.4f}, '
            f'Val Acc: {val_acc}, Test Acc: {test_acc}')
      times.append(time.time() - start)

run_fpgnn('ToxCast', 'ogbg-moltoxcast')


Extracting 	mp\mol/clintox/raw/clintox.csv.gz
Processing...
Done!
  0% 0/10 [00:04<?, ?it/s]


IndexError: ignored

In [22]:
def run_fpgnn(dataset_name, task_name):
  import torch
  import torch.nn as nn
  import torch.optim as optim
  from torch.utils.data import DataLoader
  import torch
  from torch_geometric.datasets import MoleculeNet
  import time
  from torch.utils.data import DataLoader, random_split
  from torch_geometric.data import Batch
  from tqdm import tqdm
  import numpy as np

  dataset = MoleculeNet('\tmp\mol', dataset_name).shuffle()
  ### Importing OGB dataset
  from ogb.graphproppred.dataset_pyg import PygGraphPropPredDataset
  from ogb.graphproppred import Evaluator
  evaluator = Evaluator(task_name)

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  args = Namespace(dataset_type="classification", gat_scale=1, cuda=False, dropout=0.25, nhid=128, dropout_gat=0.25, hidden_size=128, nheads=4,
                    task_num=2)
  model = FPGNN(args).to(device)
  # print(model)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

  batch_size=128
  train_size = int(0.8 * len(dataset))
  val_size = (len(dataset) - train_size) // 2
  test_size = len(dataset) - train_size - val_size
  # Split the dataset into train, validation, and test sets
  train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

  def collate(data_list):
      """
      Custom collate function to handle torch_geometric.data.data.Data instances.
      Converts a list of Data instances to a Batch.
      """
      return Batch.from_data_list(data_list)


  # Create data loaders with the custom collate function
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)

  clf_criterion = torch.nn.BCEWithLogitsLoss(torch.tensor(10))
  reg_criterion = torch.nn.MSELoss()




  def train(model, train_loader, criterion, optimizer, device):
      model.train()
      total_loss = 0.0

      epoch_iter = tqdm(train_loader, ncols=0)
      for step, data in enumerate(epoch_iter):
          data = data.to(device)
          smiles, labels = data.smiles, data.y

          optimizer.zero_grad()

          # Forward pass
          output = model(smiles)

          # Compute the loss
          is_labeled = ~torch.isnan(data.y) # Mask NaNs
          loss = criterion(output[is_labeled], data.y[is_labeled])

          # Backward pass and optimization
          loss.backward()
          # prevent exploding gradients
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
          optimizer.step()

          total_loss += loss.item()

      return total_loss / len(train_loader)

  @torch.no_grad()
  def test(model, test_loader, criterion, device):

      model.eval()

      y_true = []
      y_pred = []

      total_correct = 0
      epoch_iter = tqdm(test_loader, ncols=0)
      for step, batch in enumerate(epoch_iter):
          batch = batch.to(device)
          smiles, labels = batch.smiles, batch.y

          out = model(batch.smiles)
          output = out.cpu().numpy()

          y_true.append(batch.y.view(out.shape).detach().cpu())
          y_pred.append(out.detach().cpu())

          total_correct += int((torch.argmax(out.to(float), dim=1) == torch.argmax(batch.y.to(float), dim=1)).sum())
      y_true = torch.cat(y_true, dim = 0).numpy()
      y_pred = torch.cat(y_pred, dim = 0).numpy()
      input_dict = {"y_true": y_true, "y_pred": y_pred}
      return evaluator.eval(input_dict)

  times = []
  training_losses = []
  validation_metrics = []
  test_metrics = []
  for epoch in range(1, 6):
      start = time.time()
      train_loss = train(model, train_loader, clf_criterion, optimizer, device)
      val_acc = test(model, val_loader, clf_criterion, device)
      test_acc = test(model, test_loader, clf_criterion, device)
      training_losses.append(train_loss)
      validation_metrics.append(val_acc)
      test_metrics.append(test_acc)
      print(f'Epoch: {epoch:03d}, Loss: {train_loss:.4f}, '
            f'Val Acc: {val_acc}, Test Acc: {test_acc}')
      times.append(time.time() - start)

run_fpgnn('ClinTox', 'ogbg-molclintox')

100% 10/10 [01:12<00:00,  7.25s/it]
100% 2/2 [00:00<00:00,  2.47it/s]
100% 2/2 [00:00<00:00,  2.65it/s]


Epoch: 001, Loss: 9.0872, Val Acc: {'rocauc': 0.6692723286159803}, Test Acc: {'rocauc': 0.6616209014019232}


100% 10/10 [01:13<00:00,  7.37s/it]
100% 2/2 [00:00<00:00,  2.48it/s]
100% 2/2 [00:00<00:00,  2.63it/s]


Epoch: 002, Loss: 4.6899, Val Acc: {'rocauc': 0.673562558403981}, Test Acc: {'rocauc': 0.7107316649287452}


100% 10/10 [01:11<00:00,  7.16s/it]
100% 2/2 [00:00<00:00,  2.65it/s]
100% 2/2 [00:00<00:00,  2.78it/s]


Epoch: 003, Loss: 3.3969, Val Acc: {'rocauc': 0.6601078292295741}, Test Acc: {'rocauc': 0.7325396825396826}


100% 10/10 [01:11<00:00,  7.14s/it]
100% 2/2 [00:00<00:00,  2.64it/s]
100% 2/2 [00:00<00:00,  2.27it/s]


Epoch: 004, Loss: 3.0015, Val Acc: {'rocauc': 0.6922229821776382}, Test Acc: {'rocauc': 0.6949339589850538}


100% 10/10 [01:11<00:00,  7.14s/it]
100% 2/2 [00:00<00:00,  2.11it/s]
100% 2/2 [00:00<00:00,  2.09it/s]

Epoch: 005, Loss: 2.7973, Val Acc: {'rocauc': 0.6991033933416646}, Test Acc: {'rocauc': 0.7272216429150736}
